In [1]:

# 무위험수익률, 수익률,공분산으로 평균-분산 프런티어 계산
def solveFrontier(R, C, rf):
    def fitness(W, R, C, r):
        
        # 주어진 수익률에서 분산을 최소로 하는 비중 계산
        mean = sum(R * W)
        var = np.dot(np.dot(W, C), W)
        
        # 최적화 제약조건 페널티
        penalty = 100 * abs(mean - r)             
        return var + penalty

    frontier_mean, frontier_var = [], [] # 효율적 투자선을 구성하는 평균-분산을 돌려줄 리스트
    n = len(R)                           # 투자자산 갯수
    
    # 수익률 최저~최대 사이를 반복한다
    for r in linspace(min(R), max(R), num=20):  # Iterate through the range of returns on Y axis
        # 최적화 함수에 전달할 초기값으로 동일비중으로 시작
        W = np.ones([n]) / n 
        
        # 최적화 함수에 전달할 제약조건을 미리 준비한다
        b_ = [(0, 1) for i in range(n)]                      # 범위조건: 각 구성 자산의 투자 비중은 0~100% 사이
        c_ = ({'type': 'eq', 'fun': lambda W: sum(W) - 1.})  # 제약조건: 전체 투자비중은 100%이다.
        
        # 최적화 함수 minimize()은 최적화할 함수와 최적화를 시작할 초깃값을 인수로 받는다. 
        # fitness가 최적화할 함수이다.
        optimized = scipy.optimize.minimize(fitness, W, (R, C, r), method='SLSQP', constraints=c_, bounds=b_)
        if not optimized.success:
            raise BaseException(optimized.message) # 최적화 실패 경우
        # 최적 포트폴리오 수익률과 분산 추가  [x,y] = [optimized.x, r]
        # 효율적 프런티어에 포트폴리오 하나를 추가
        frontier_mean.append(r)
        frontier_var.append(get_var(optimized.x, C))
    return array(frontier_mean), array(frontier_var)